# ERK-KTR Full FOV Stimulation Pipeline

## System Init

### Load pymmcore and required python libraries

In [1]:
import os
os.environ['QT_LOGGING_RULES'] = '*.debug=false; *.warning=false' # Fix to suppress PyQT warnings from napari-micromanager when running in a Jupyter notebook

from fov import FOV
from useq import MDAEvent
import pandas as pd
import numpy as np
import random
import napari
import pymmcore_plus
from napari_micromanager import MainWindow

from utils import create_folders
from useq._mda_event import SLMImage

mmc = pymmcore_plus.CMMCorePlus()

### Device Specific Init for Niesen Microscope

In [2]:
mmc.loadSystemConfiguration("E:\\pertzlab_mic_configs\\micromanager\\Niesen\\Ti2CicercoConfig_w_DMD_w_TTL.cfg")

### The following libraries are used to wake up the Lumencor laser
import requests
import threading
import time

class WakeUpLaser:
    def __init__(self, lumencore_ip="192.168.201.200"): 
        self.ip = lumencore_ip
        self.last_wakeup = 0
        self.is_running = False
        
    def wakeup_laser(self):
        url = f"http://{self.ip}/service/?command=WAKEUP"
        requests.get(url)
        
    
    def run(self, wait_for_warmup=False):
        self.is_running = True
        self.thread = threading.Thread(target=self._keep_alive)
        self.thread.start()
        if wait_for_warmup:
            time.sleep(15)

    def _keep_alive(self):
        while self.is_running:
            if time.time() - self.last_wakeup > 60:
                self.wakeup_laser()
                self.last_wakeup = time.time()
            time.sleep(3)
    def stop(self):
        self.is_running = False
        self.thread.join()
        
wl = WakeUpLaser()
wl.wakeup_laser()

slm_dev = mmc.getSLMDevice()
slm_width = mmc.getSLMWidth(slm_dev)
slm_height = mmc.getSLMHeight(slm_dev)

event_slm_on = MDAEvent(slm_image=SLMImage(data=True))
mmc.mda.run([event_slm_on])# to only have fov of DMD 
mmc.setROI(150, 150, 1900, 1900)

DMD_CHANNEL_GROUP = "WF_DMD"
DMD_CALIBRATION_PROFILE = {"channel_group": "WF_DMD", "channel_config": "CyanStim", "device_name": "LedDMD", "property_name": "Cyan_Level", "power": 100}
mmc.setChannelGroup(channelGroup=DMD_CHANNEL_GROUP)

[03/05/25 16:46:49] INFO     MDA Started: GeneratorMDASequence()                                     ]8;id=219171;file://c:\Users\Niesen\.conda\envs\pymmcore_w_stardist_cpu\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=704215;file://c:\Users\Niesen\.conda\envs\pymmcore_w_stardist_cpu\lib\site-packages\pymmcore_plus\mda\_runner.py#376\376]8;;\

                    INFO     slm_image=SLMImage()                                                    ]8;id=847604;file://c:\Users\Niesen\.conda\envs\pymmcore_w_stardist_cpu\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=531815;file://c:\Users\Niesen\.conda\envs\pymmcore_w_stardist_cpu\lib\site-packages\pymmcore_plus\mda\_runner.py#337\337]8;;\

                    INFO     MDA Finished: GeneratorMDASequence()                                    ]8;id=515952;file://c:\Users\Niesen\.conda\envs\pymmcore_w_stardist_cpu\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=153506;file://c:\Users\Niesen\.conda\envs\pymmcore_w_stardist_cpu\lib\site-packages\pymmcore_plus\mda\_runner.py#463\463]8;;\

## GUI - Napari Micromanager

### Load GUI

In [3]:
### Base GUI ###
viewer = napari.Viewer()
mm_wdg = MainWindow(viewer)
viewer.window.add_dock_widget(mm_wdg)

In [4]:
### Add MDA widget for FOV selection ###
from pymmcore_widgets.mda import MDAWidget
mdawidget = MDAWidget(mmcore = mmc)
viewer.window.add_dock_widget(mdawidget)

### Functions to break and re-connect link with GUI if manually broken

The following functions can be used to manually interrupt to connection between the GUI and the running rtm-pymmcore script. However, normally you don't need to execute them. 

In [10]:
### Break connection
# mm_wdg._core_link.cleanup()

In [5]:
### Manually reconnect pymmcore with napari-micromanager
from napari_micromanager._core_link import CoreViewerLink
mm_wdg._core_link = CoreViewerLink(viewer, mmc)

## Create a DF with all planned acquisitions and stimulations

### Settings for Experiment

In [30]:
import pandas as pd
import os
from utils import create_folders
from fov import FOV
import numpy as np
import random

In [75]:
N_TIMESTEPS = 120
df_acquire = pd.DataFrame(columns=['fov', 'timestep', 'time','time_experiment', 'treatment', 'acquired','stim', 'channels', 'channel_stim'])

base_path = "\\\\izbkingston.unibe.ch\\imaging.data\\mic01-imaging\\Alex\\pymmcore\\"
experiment_name = "2025-03-18_optoTIAM_migration_percentage_Niesen_miRFP_mcherry"
path  = os.path.join(base_path, experiment_name)

# create_folders(path,['stim','raw','labels','stim_mask','tracks','labels_rings','particles'])

time_between_frames = 10 #time in seconds between frames
time_per_fov = 5 #time in seconds per fov

timesteps = range(N_TIMESTEPS)  
channels = ['Red', 'Green'] #channel for segmentation first
channels_exposure = [200, 200]

# take values from UI, if loaded
# intensity_red_laser = mmc.getProperty("Laser", "RED_Intensity")
# intensity_green_laser = mmc.getProperty("Laser", "GREEN_Intensity")

# if intensity_red_laser != str(0) and intensity_green_laser != str(0):
#     channels_power = [intensity_red_laser, intensity_green_laser]
# else:
#     channels_power = [120, 120]
#     for channel, power in zip(channels, channels_power):
#         mmc.setProperty("Laser", f"{channel.upper()}_Intensity", power)


stim_exposures = [100] # list of possible exposures in ms
stim_timesteps = [list(range(0,15))]  # list of timesteps for stimulation, if e.g. double stimulation in frame 0 and 1 is needed write [[0,1]]
stim_percentages = [0.2, 0.3]
stim_profiles = [{"device_name": "LedDMD", "property_name": "Cyan_Level", "power": 10, "channel": "CyanStim"}]
stim_treatments = [{"stim_profile": stim_profile, 
                   "stim_exposure": stim_exposure, 
                   "stim_timestep": stim_timestep, 
                   "stim_cell_percentage": stim_cell_percentage} 
                  for stim_profile in stim_profiles 
                  for stim_exposure in stim_exposures 
                  for stim_timestep in stim_timesteps
                  for stim_cell_percentage in stim_percentages]
cell_lines = ["miRFP", "mCherry"]
n_fovs_per_cell_line = 5
# data_mda_fovs = None
stim_treatments



[{'stim_profile': {'device_name': 'LedDMD',
   'property_name': 'Cyan_Level',
   'power': 10,
   'channel': 'CyanStim'},
  'stim_exposure': 100,
  'stim_timestep': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
  'stim_cell_percentage': 0.2},
 {'stim_profile': {'device_name': 'LedDMD',
   'property_name': 'Cyan_Level',
   'power': 10,
   'channel': 'CyanStim'},
  'stim_exposure': 100,
  'stim_timestep': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
  'stim_cell_percentage': 0.3}]

In [26]:
import matplotlib.pyplot as plt
plt.figure()
data_mda_fovs = mdawidget.value().stage_positions
i = 0
for fov_x_y in data_mda_fovs:
    plt.scatter(fov_x_y.x, fov_x_y.y, label=f'{fov_x_y.x},{fov_x_y.y}')
    plt.text(fov_x_y.x, fov_x_y.y, f'{i}')
    i+=1
plt.show()

NameError: name 'mdawidget' is not defined

<Figure size 640x480 with 0 Axes>

### Map Experiment to FOVs

#### If FOVs already saved - Reload them from file

In [70]:
import json
# file = os.path.join(path, "fovs.json")
file = "fovs.json"
with open(file, "r") as f:
    data_mda_fovs = json.load(f)

Only select one of the following two code blocks. If you autogenerated FOVs using the wellplate option of the MDA widget, then use the first code block, else the second. 

#### MDA Widget was used in wellplate mode with autogenerated FOVs

#### FOVs were manually selected using MDA widget

In [76]:
fovs:list[FOV] = []
if data_mda_fovs is None:
    data_mda_fovs = mdawidget.value().stage_positions
n_fovs = len(data_mda_fovs)
n_stim_treatments = len(stim_treatment)
n_fovs_per_stim_condition = n_fovs //  len(np.unique(cell_lines))

j = 0
for stim_treatment in stim_treatments: 
    for i, row in enumerate(data_mda_fovs):
        row = dict(row)
        if len(cell_lines) == 1:
            cell_line = cell_lines[0]
        else:
            cell_line = cell_lines[i // n_fovs_per_cell_line]
        fov = FOV(pos=(row["x"], row["y"]),
                index=j,
                name=str(i),
                path=path,
                metadata={"cell_line": cell_line},
                treatment=stim_treatment,
                )
        fovs.append(fov)
        j +=1

### Use FOVs to generate dataframe for acquisition

In [77]:
n_fovs_simultaneously = time_between_frames // time_per_fov
start_time = 0

dfs = []
for fov in fovs:
    fov_group = fov.index // n_fovs_simultaneously
    start_time = fov_group * time_between_frames * len(timesteps)

    for timestep in timesteps:
        if fov.metadata['cell_line'] == "mCherry":
            channels_power = [120]
            channels = ['Green']
            channels_exposure = [200]
        if fov.metadata['cell_line'] == "miRFP":
            channels_power = [120]
            channels = ['Red']
            channels_exposure = [200]

        new_row = { 'fov_object': fov,
                    'fov':fov.index,
                    'name':fov.name,
                    'timestep': timestep,
                    'time': start_time + timestep*time_between_frames,
                    'treatment': fov.treatment,
                    'metadata': fov.metadata,
                    'stim': timestep in fov.treatment['stim_timestep'] and fov.treatment["stim_exposure"] != 0, # not really important, only for visualisation
                    'channels': channels,
                    'channels_exposure':channels_exposure,
                    'channel_power': channels_power,
                    'fname' : f'{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}',
                    }
        dfs.append(new_row)

df_acquire = pd.DataFrame(dfs)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', True)
df_acquire = df_acquire.sort_values(by=['time', 'fov'])
print(f"Total Experiment Time: {df_acquire['time'].max()/3600}h")
df_acquire


Total Experiment Time: 3.3305555555555557h


,fov_object,fov,name,timestep,time,treatment,metadata,stim,channels,channels_exposure,channel_power,fname
0,<fov.FOV object at 0x000002187F6710D0>,0,0,0,0,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'miRFP'},True,[Red],[200],[120],000_00000
120,<fov.FOV object at 0x000002187F670A10>,1,1,0,0,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'miRFP'},True,[Red],[200],[120],001_00000
1,<fov.FOV object at 0x000002187F6710D0>,0,0,1,10,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'miRFP'},True,[Red],[200],[120],000_00001
121,<fov.FOV object at 0x000002187F670A10>,1,1,1,10,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'miRFP'},True,[Red],[200],[120],001_00001
2,<fov.FOV object at 0x000002187F6710D0>,0,0,2,20,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'miRFP'},True,[Red],[200],[120],000_00002
...,...,...,...,...,...,...,...,...,...,...,...,...
2397,<fov.FOV object at 0x000002187F671910>,19,9,117,11970,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'mCherry'},False,[Green],[200],[120],019_00117
2278,<fov.FOV object at 0x000002187F671850>,18,8,118,11980,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'mCherry'},False,[Green],[200],[120],018_00118
2398,<fov.FOV object at 0x000002187F671910>,19,9,118,11980,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'mCherry'},False,[Green],[200],[120],019_00118
2279,<fov.FOV object at 0x000002187F671850>,18,8,119,11990,"{'stim_profile': {'device_name': 'LedDMD', 'pr...",{'cell_line': 'mCherry'},False,[Green],[200],[120],018_00119


## Run experiment

In [ ]:
%load_ext autoreload
%autoreload 2
from add_frame import ImageProcessingPipeline
from segmentation.imaging_server import SegmentatorImagingServerKit
from stimulation.percentage_of_cell import StimPercentageOfCell    
from controller import Controller, Analyzer
from tracking.trackpy import TrackerTrackpy
from dmd import DMD
from queue import Queue

try: 
    mm_wdg._core_link.cleanup()
except:
    pass

segmentator = SegmentatorImagingServerKit()
stimulator = StimPercentageOfCell()
tracker = TrackerTrackpy()
dmd = DMD(mmc, DMD_CALIBRATION_PROFILE)

pipeline = ImageProcessingPipeline(segmentator,stimulator,tracker)
analyzer = Analyzer(pipeline)
queue = Queue()
controller = Controller(analyzer, mmc, queue, dmd)
wl.run(wait_for_warmup=True)
controller.run(df_acquire)
wl.stop()